# Batch Image Downloader for jav.ink of Graphis Collection

In [1]:
from user_agent import generate_user_agent
from bs4 import BeautifulSoup
from random import randint
from pathlib import Path
import urllib.parse
import requests
import shutil
import os

In [2]:
##  https://www.jav.ink/category/graphis-collection-2002-2018/
##  https://www.jav.ink/category/graphis-collection-2002-2018/page/2/
##  https://www.jav.ink/category/graphis-collection-2002-2018/page/85/
##  https://www.jav.ink/graphis-collection-2002-2018/amin-niina-%e6%96%b0%e5%90%8d%e3%81%82%e3%81%bf%e3%82%93/
_url = "https://www.jav.ink/category/graphis-collection-2002-2018/"

_outputPath = "../Dataset/Photos/Adult/jav.ink/graphis-collection-2002-2018/"

In [3]:
def downloadAlbum(url):

    if (len(url) < 10):
        return

    ##  If folder already exists, means already downloaded, skip it
    path = Path(url)
    pathName = urllib.parse.unquote(path.name)
    filePath = os.path.join(_outputPath, pathName)
    exist = os.path.exists(filePath)
    if (exist == True):
        return

    ##  Create path
    os.makedirs(filePath)

    userAgent = generate_user_agent()
    headers = {"user-agent":userAgent, "referer":_url}
    httpRequest = requests.get(url=url, headers=headers)
    soup = BeautifulSoup(httpRequest.text, "html.parser")
    print(httpRequest.url)

    objectList = soup.findAll("a")
    if (objectList is None):
        return

    for imageObject in objectList:

        imageURL = imageObject.get("href")
        if (imageURL is None):
            print("### Image URL not found...")
            print(soup)
            finish = True
            return
        
        ##  Check if the link I want
        if "uploads" not in imageURL:
            continue

        ##  Download image
        imageURL = imageURL.rstrip()
        imageFilename = imageURL.split("/")[-1]
        
        array = imageFilename.split(".jpg")
        imageFilename = array[0]+".jpg"

        #print("Downloading "+imageURL+"...")
        imageURL = imageURL+"?v="+str(randint(1, 100))
        httpRequest = requests.get(imageURL, headers=headers, stream=True)
        if (httpRequest.status_code != 200):
            print("### Unable to download image...\n")
            continue

        ##  Set true to prevent image size zero
        httpRequest.raw.decode_content = True

        ##  Save image
        path = os.path.join(filePath, imageFilename)
        with open(path, "wb") as file:

            shutil.copyfileobj(httpRequest.raw, file)


In [4]:
finish = False
albumURL = _url
userAgent = generate_user_agent()
for i in range(100):

    page = i+1
    print("Album page #"+str(page)+"...")

    url = albumURL
    if (page > 1):
        url = albumURL+"page/"+str(page)+"/"
    
    headers = {"user-agent":userAgent, "referer":_url}
    httpRequest = requests.get(url=url, headers=headers)
    soup = BeautifulSoup(httpRequest.text, "html.parser")

    albumList = soup.findAll("a")
    if (albumList is None):
        finish = True
        break

    finish = True
    for linkObject in albumList:

        ##  Check if it is Graphis album
        url = linkObject.get("href")
        array = url.split("graphis-collection-2002-2018/")
        if (len(array) < 2):
            continue
        
        if (array[0].startswith("https://www.jav.ink/") == False):
            continue

        if (len(array[1]) < 10):
            continue
        
        text = linkObject.text
        if (len(text) < 4):
            continue

        ##  Yes, Graphis album
        #print(url)
        downloadAlbum(url)
        finish = False

    if (finish == True):
        break

print("Finish all!")

Album page #1...
Album page #2...
https://www.jav.ink/graphis-collection-2002-2018/rion%E3%83%95%E3%82%A1%E3%83%BC%E3%82%B9%E3%83%88%E5%86%99%E7%9C%9F%E9%9B%86%E3%80%8Emy-home%E3%80%8F/
https://www.jav.ink/graphis-collection-2002-2018/an-tsujimoto-%E8%BE%BB%E6%9C%AC%E6%9D%8F-oriental-lily/
Album page #3...
https://www.jav.ink/graphis-collection-2002-2018/fumika-hatsuno-%E5%88%9D%E4%B9%83%E3%81%B5%E3%81%BF%E3%81%8B/
Album page #4...
https://www.jav.ink/graphis-collection-2002-2018/%E9%BA%97%E3%81%97%E3%81%AE%E7%BE%8E%E4%BA%BA%E5%A5%B3%E6%95%99%E5%B8%AB%E3%80%81%E7%94%B7%E5%AC%B2%E3%82%8A%E3%80%82-%E5%90%89%E6%B2%A2%E6%98%8E%E6%AD%A9/


KeyboardInterrupt: 